# Juan Carlos Perez Ramirez
## Procesamiento de Lenguaje Natural
## Tarea 6: Hierarchical Attention

### Corpus limpio generado
Los archivos de entrenamiento y de validacion se encuentran en https://drive.google.com/drive/folders/1WfVIBr6H1YLqSsuGzQ8mFGvKg-xCBLu_?usp=sharing

### Preprocesamiento
Se construye un archivo csv con la estructura tweet1\</s\>tweet2\</s\>tweet3\</s\>nacionalidad, sustituyendo las urls por el token \<url\> y las menciones por USER.

In [1]:
import os
import xml.etree.ElementTree as ET
import re

# Rutas de entrada y salida
split = "val"
dir = "es_" + split
path = "../../corpus/2025AuthorProfiling_Train_Val/"
truth = path + dir + '/truth.txt'  # Tu archivo con los identificadores
xml_dir = path + dir  # Carpeta donde están los archivos XML
docs_file = path + split + '.csv'

# Abrimos los archivos de salida
with open(docs_file, 'w', encoding='utf-8') as f_textos, \
     open(truth, 'r', encoding='utf-8') as f_in:

    for linea in f_in:
        partes = linea.strip().split(":::")
        if len(partes) != 3:
            continue  # Saltamos líneas mal formateadas

        identificador = partes[0]
        nacionalidad = partes[2]
        archivo_xml = os.path.join(xml_dir, f"{identificador}.xml")

        if not os.path.exists(archivo_xml):
            print(f"Archivo {archivo_xml} no encontrado, saltando.")
            continue

        try:
            tree = ET.parse(archivo_xml)
            root = tree.getroot()
            documentos = root.find('documents')

            # Guardamos todos los tweets en una lista
            tweets = []
            for doc in documentos.findall('document'):
                texto = doc.text.strip()
                texto = re.sub(r"http\S+|www\S+|https\S+", "<url>", texto)  # sustituye URLs por "<url>"
                texto = re.sub(r"@\w+", "USER", texto)
                texto = texto.replace('\n', ' ')  # elimina saltos de línea dentro del tweet
                tweets.append(texto)

            if tweets:
                linea_usuario = " </s> ".join(tweets) + " </s>" + nacionalidad + "\n"
                f_textos.write(linea_usuario)

        except Exception as e:
            print(f"Error procesando {archivo_xml}: {e}")



In [2]:
import pandas as pd
import pickle
import numpy as np
import nltk
nltk.download('punkt')
from tqdm.auto import tqdm
import copy

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
import torch.nn.functional as F

from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /home/juancho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
class author_profiling_dataset(Dataset):
    def __init__(self, split):
        super(Dataset, self).__init__()
        self.load_data(split)
        self.vocab, self.emb_mat = self.load_vocab_embeddings()
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        tweet_texts = self.data.iloc[index]['tweets']  # Lista de strings
        label = self.data.iloc[index]['target']

        tweet_word_ids = []
        tweet_tokens = []

        for tweet in tweet_texts:
            words, word_ids = self.preprocessed_text(tweet)
            
            # Si está vacío, lo ignoramos
            if len(word_ids) == 0:
                continue
            
            tweet_word_ids.append(word_ids)
            tweet_tokens.append(words)

        # Si todos los tweets estaban vacíos, agregamos uno con [UNK] para evitar errores
        if len(tweet_word_ids) == 0:
            tweet_word_ids.append([1])  # [UNK]
            tweet_tokens.append(['[UNK]'])

        return tweet_word_ids, label, tweet_tokens

        
    def preprocessed_text(self, text):
        words = nltk.TweetTokenizer().tokenize(text)
        
        # Convertir a minúsculas y filtrar si hace falta (opcional)
        words = [w.lower() for w in words]

        word_ids = [
            self.vocab[word] if word in self.vocab else 1  # [UNK]
            for word in words
        ]

        return words, word_ids


    def load_data(self, split):
        textos = []
        etiquetas = []

        with open(f'{split}.csv', 'r', encoding='utf-8') as f:
            for line in f:
                if "</s>" in line:
                    l = line.strip().split("</s>")
                    tweet_list = l[:-1]
                    tweet_list = [t.strip() for t in tweet_list if t.strip()]
                    textos.append(tweet_list)
                    etiquetas.append(l[-1])


        self.data = pd.DataFrame({'tweets': textos, 'target': etiquetas})
        self.label_map = {l: i for i, l in enumerate(sorted(set(etiquetas)))}
        self.data['target'] = self.data['target'].map(self.label_map)

        print(f"Clases: {self.label_map}")


    def load_vocab_embeddings(self):
        '''Embeddings preentrenados en twitter.
           emb_mat: Matriz de embeddings. Un vector de tamaño 200 para cada palabra del vocabulario.
           vocab: Diccionario, asigna a cada palabra su renglón correspondiente en la matriz de embeddings.
        '''
        embeddings_list = []
        self.vocab_dict = {}
        vocab = {}
        with open('word2vec_col.txt', 'r') as f:
            for i, line in enumerate(f):
                if i!=0:
                    values = line.split()
                    self.vocab_dict[i+1] = values[0]
                    vocab[values[0]] = i+1
                    vector = np.asarray(values[1:], "float32")
                    embeddings_list.append(vector)
        embeddings_list.insert(0,np.mean(np.vstack(embeddings_list), axis=0))
        embeddings_list.insert(0,np.zeros(100))
        self.vocab_dict[0] = '[PAD]'
        self.vocab_dict[1] = '[UNK]'
        vocab['[PAD]'] = 0
        vocab['[UNK]'] = 1
        emb_mat = np.vstack(embeddings_list)

        return vocab, emb_mat

    def get_weights(self):
        counts = self.data['target'].value_counts().sort_index()
        maxi = counts.max()
        weights = [maxi / count for count in counts]
        return torch.tensor(weights, dtype=torch.float32)


    def collate_fn(self, batch):
        """
        batch: lista de tuplas (tweet_word_ids, label, tweet_tokens)
            - tweet_word_ids: List[List[int]] (lista de tweets, cada uno como lista de ids)
            - label: int
            - tweet_tokens: List[List[str]] (opcional, para debugging)
        """

        doc_lengths = []             # batch_size (número de tweets por autor)
        tweet_lengths_batch = []     # batch_size x num_tweets (longitudes de palabras)
        tweet_sequences_batch = []   # batch_size x num_tweets x num_words

        labels = []                  # batch_size
        tokens = []                  # batch_size x num_tweets x num_words

        for tweet_ids, label, tweet_tokens in batch:
            # Rellenar tweets vacíos con [UNK] (id=1)
            tweet_tensors = [
                torch.tensor(t if len(t) > 0 else [1]) for t in tweet_ids
            ]
            tweet_lengths = []
            cleaned_tweets = []

            for t in tweet_ids:
                if len(t) == 0:
                    cleaned_tweets.append(torch.tensor([1]))  # [UNK]
                    tweet_lengths.append(1)
                else:
                    cleaned_tweets.append(torch.tensor(t))
                    tweet_lengths.append(len(t))

            tweet_tensors = cleaned_tweets


            tweet_sequences_batch.append(tweet_tensors)
            tweet_lengths_batch.append(tweet_lengths)
            doc_lengths.append(len(tweet_ids))
            labels.append(label)
            tokens.append(tweet_tokens)

        max_tweets = max(doc_lengths)
        max_words = max([max(l) if l else 0 for l in tweet_lengths_batch])

        # Padding a nivel de palabra
        for i in range(len(tweet_sequences_batch)):
            tweet_tensors = tweet_sequences_batch[i]
            padded_tweets = pad_sequence(tweet_tensors, batch_first=True, padding_value=0)  # (num_tweets_i, max_words_i)

            # Pad a max_words por tweet
            if padded_tweets.shape[1] < max_words:
                pad_width = max_words - padded_tweets.shape[1]
                padding = torch.zeros((padded_tweets.shape[0], pad_width), dtype=torch.long)
                padded_tweets = torch.cat([padded_tweets, padding], dim=1)

            # Pad a max_tweets por autor
            if padded_tweets.shape[0] < max_tweets:
                pad_height = max_tweets - padded_tweets.shape[0]
                padding = torch.zeros((pad_height, max_words), dtype=torch.long)
                padded_tweets = torch.cat([padded_tweets, padding], dim=0)

            tweet_sequences_batch[i] = padded_tweets

        tweet_tensor = torch.stack(tweet_sequences_batch)   # (batch_size, max_tweets, max_words)
        label_tensor = torch.tensor(labels)
        doc_lengths = torch.tensor(doc_lengths)
        tweet_lengths_batch = [tl + [0]*(max_tweets - len(tl)) for tl in tweet_lengths_batch]
        tweet_lengths_tensor = torch.tensor(tweet_lengths_batch)

        return tweet_tensor, doc_lengths, tweet_lengths_tensor, label_tensor, tokens


In [4]:
import pandas as pd
import pickle
import numpy as np
import nltk
nltk.download('punkt')
from tqdm.auto import tqdm
import copy

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
import torch.nn.functional as F

from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /home/juancho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Clasificador con atencion jerarquica

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.proy = nn.Linear(hidden_size * 2, hidden_size)
        self.u_w = nn.Parameter(torch.randn(hidden_size))

    def forward(self, h_it, mask=None):
        """
        h_it: tensor (batch_size, seq_len, hidden_size*2) — anotaciones de palabras (o tweets)
        mask: tensor (batch_size, seq_len) — 0 donde hay padding
        """
        u_it = torch.tanh(self.proy(h_it))
        alpha_it = torch.matmul(u_it, self.u_w)
        
        if mask is not None:
            mask = mask.to(alpha_it.device)
            alpha_it = alpha_it.masked_fill(mask == 0, -1e9)
            
        alpha_it = F.softmax(alpha_it, dim=1)
        s_i = torch.sum(h_it * alpha_it.unsqueeze(-1), dim=1)
        return s_i, alpha_it


In [ ]:
class HAN(nn.Module):
    def __init__(self, num_classes, emb_mat, hidden_size=128):
        super().__init__()
        
        vocab_size, emb_dim = emb_mat.shape
        
        # Embeddings
        with torch.no_grad():
            self.embeddings = nn.Embedding.from_pretrained(
                torch.tensor(emb_mat, dtype=torch.float32),
                freeze=True
            )

        # WORD
        self.word_encoder = nn.GRU(
            input_size=emb_dim,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True
        )
        self.word_attention = Attention(hidden_size)

        # SENTENCE
        self.sent_encoder = nn.GRU(
            input_size=hidden_size * 2,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True
        )
        self.sent_attention = Attention(hidden_size)

        # Clasificador
        self.classifier = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x, doc_lengths, tweet_lengths):
        """
        x: (batch_size, max_tweets, max_words)
        doc_lengths: número real de tweets por autor (batch_size,)
        tweet_lengths: número real de palabras por tweet (batch_size, max_tweets)
        """

        batch_size, max_tweets, max_words = x.size()

        # WORD-LEVEL
        x = x.view(-1, max_words)
        tweet_lengths = tweet_lengths.view(-1)
        tweet_lengths = tweet_lengths.clamp(min=1)
        mask_words = (x != 0)

        x_emb = self.embeddings(x)

        packed_words = nn.utils.rnn.pack_padded_sequence(
            x_emb, tweet_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        h_it, _ = self.word_encoder(packed_words)
        h_it, _ = nn.utils.rnn.pad_packed_sequence(h_it, batch_first=True)

        s_i, alpha_it = self.word_attention(h_it, mask_words)

        # Guarda atención de palabras
        self.word_attn_scores = alpha_it.view(batch_size, max_tweets, -1)  # shape: (batch_size, max_tweets, max_words)

        # SENTENCE-LEVEL
        s_i = s_i.view(batch_size, max_tweets, -1)
        mask_sents = (doc_lengths.unsqueeze(1) > torch.arange(max_tweets).to(doc_lengths.device)).int()

        h_i, _ = self.sent_encoder(s_i)
        v, alpha_i = self.sent_attention(h_i, mask_sents)

        # Guarda atención de tweets
        self.sent_attn_scores = alpha_i  # shape: (batch_size, max_tweets)

        # CLASIFICACIÓN
        logits = self.classifier(v)
        return logits


In [87]:
def eval_model(model, dataloader, criterion, device, attention_level='sent'):
    with torch.no_grad():
        model.eval()
        losses = []
        preds = torch.empty(0).long()
        targets = torch.empty(0).long()
        scores_list = []
        words_list = []
        pred_list = []

        for data in tqdm(dataloader):
            torch.cuda.empty_cache()
            seq, seq_len, tweet_lens, labels, words = data
            seq, labels = seq.to(device), labels.to(device)

            output = model(seq, seq_len, tweet_lens)
            loss = criterion(output, labels)
            losses.append(loss.item())

            predictions = output.argmax(1).cpu()
            preds = torch.cat([preds, predictions], dim=0)
            targets = torch.cat([targets, labels.cpu()], dim=0)

            # Elegimos qué atención guardar
            if attention_level == 'sent' and hasattr(model, "sent_attn_scores"):
                scores = model.sent_attn_scores.cpu().tolist()
            elif attention_level == 'word' and hasattr(model, "word_attn_scores"):
                scores = model.word_attn_scores.cpu().tolist()
            else:
                scores = []

            scores_list += scores
            pred_list += predictions.tolist()
            words_list += words

        model.train()
        preds = preds.numpy()
        targets = targets.numpy()
        f1 = f1_score(targets, preds, average='macro')

        return np.mean(losses), f1, scores_list, words_list, pred_list



In [8]:
batch_size=32

In [27]:
train_dataset = author_profiling_dataset('train')
test_dataset = author_profiling_dataset('val')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn = train_dataset.collate_fn, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn = test_dataset.collate_fn, shuffle=False)

Clases: {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}
Clases: {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}


In [58]:
lr = 0.0001
epochs = 20
device = torch.device('cuda')
weight_decay=0.0001
beta1=0
beta2=0.999

In [74]:
num_classes = len(train_dataset.label_map.keys())
model = HAN(num_classes=num_classes, emb_mat=train_dataset.emb_mat).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay, betas = (beta1, beta2))
weight = train_dataset.get_weights().to(device)
criterion = nn.CrossEntropyLoss(weight = weight)

In [89]:
best_val_f1 = 0
for epoch in range(epochs):
    model.train()
    for data in tqdm(train_dataloader):
        optimizer.zero_grad()
        
        tweet_tensor, doc_lengths, tweet_lengths_tensor, labels, _ = data
        tweet_tensor = tweet_tensor.to(device)
        labels = labels.to(device)
        doc_lengths = doc_lengths.to(device)
        tweet_lengths_tensor = tweet_lengths_tensor.to(device)

        output = model(tweet_tensor, doc_lengths, tweet_lengths_tensor)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    torch.cuda.empty_cache()

    model.eval()
    with torch.no_grad():
        train_loss, train_f1, train_scores, train_words, train_pred = eval_model(
            model, train_dataloader, criterion, device, attention_level='sent'
        )
        val_loss, val_f1, _, _, _ = eval_model(
            model, test_dataloader, criterion, device, attention_level='sent'
        )

    print(f'epoch: {epoch}')
    print(f'train_loss: {train_loss:.5f} | val_loss: {val_loss:.5f} | train_f1: {train_f1:.5f} | val_f1: {val_f1:.5f}')
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_state_dict = copy.deepcopy(model.state_dict())


  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

epoch: 0
train_loss: 0.27891 | val_loss: 0.93510 | train_f1: 0.91974 | val_f1: 0.75143


  0%|          | 0/105 [00:00<?, ?it/s]

...

epoch: 18
train_loss: 0.07970 | val_loss: 1.00086 | train_f1: 0.98692 | val_f1: 0.77457


  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

epoch: 19
train_loss: 0.06925 | val_loss: 0.99165 | train_f1: 0.98990 | val_f1: 0.77350


In [90]:
model.load_state_dict(best_state_dict)
train_loss, train_f1, train_scores, train_words, train_pred = eval_model(
            model, train_dataloader, criterion, device, attention_level='sent'
        )
test_loss, test_f1, _, _, _ = eval_model(
            model, test_dataloader, criterion, device, attention_level='sent'
        )
print('train_loss: %5f | train_f1: %5f'%(train_loss, train_f1))
print('test_loss: %5f | test_f1: %5f'%(test_loss, test_f1)) 

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

train_loss: 0.212829 | train_f1: 0.942975
test_loss: 0.910213 | test_f1: 0.775427


Desde la primera epoca se observa un valor de F1 del 91\% en el conjunto de entrenamiento, y con un maximo de 94%, indicando que se trata de una arquitectura muy poderosa para este tipo de tarea; sin embargo, su costo computacional tanto de tiempo como de memoria es demasiado alto, de manera que el uso de modelos mas simples (como BoW) continua siendo atractivo

# Visualizacion

In [83]:
att = np.linspace(0,1,50)
p = [' ']*50
s = colorize(p, att)
# to display in ipython notebook
display(HTML(s))

/tmp/ipykernel_9089/2656899777.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = matplotlib.cm.get_cmap('Reds')


## A nivel palabra

In [95]:
best_val_f1 = 0
for epoch in range(epochs):
    model.train()
    for data in tqdm(train_dataloader):
        optimizer.zero_grad()
        
        tweet_tensor, doc_lengths, tweet_lengths_tensor, labels, _ = data
        tweet_tensor = tweet_tensor.to(device)
        labels = labels.to(device)
        doc_lengths = doc_lengths.to(device)
        tweet_lengths_tensor = tweet_lengths_tensor.to(device)

        output = model(tweet_tensor, doc_lengths, tweet_lengths_tensor)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    torch.cuda.empty_cache()

    model.eval()
    with torch.no_grad():
        train_loss, train_f1, train_scores, train_words, train_pred = eval_model(
            model, train_dataloader, criterion, device, attention_level='word'
        )
        val_loss, val_f1, _, _, _ = eval_model(
            model, test_dataloader, criterion, device, attention_level='word'
        )

    print(f'epoch: {epoch}')
    print(f'train_loss: {train_loss:.5f} | val_loss: {val_loss:.5f} | train_f1: {train_f1:.5f} | val_f1: {val_f1:.5f}')
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_state_dict = copy.deepcopy(model.state_dict())


  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

epoch: 0
train_loss: 0.19691 | val_loss: 0.94329 | train_f1: 0.94866 | val_f1: 0.75829


  0%|          | 0/105 [00:00<?, ?it/s]

...

epoch: 18
train_loss: 0.04949 | val_loss: 1.02244 | train_f1: 0.99227 | val_f1: 0.77437


  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

epoch: 19
train_loss: 0.04355 | val_loss: 1.03514 | train_f1: 0.99316 | val_f1: 0.77790


In [97]:
def colorize_words_per_tweet(tweet_list, attn_matrix):
    """
    tweet_list: List[List[str]] → Lista de tweets (cada uno como lista de palabras)
    attn_matrix: List[List[float]] → Atenciones por palabra para cada tweet
    """
    cmap = matplotlib.cm.get_cmap('Reds')
    template = '<div style="margin:4px 0;">{}</div>'
    word_template = '<span style="color: black; background-color: {}">&nbsp;{}&nbsp;</span>'
    
    output = ""
    for tweet_words, word_scores in zip(tweet_list, attn_matrix):
        line = ""
        for word, score in zip(tweet_words, word_scores):
            color = matplotlib.colors.rgb2hex(cmap(score)[:3])
            line += word_template.format(color, word)
        output += template.format(line)
    return output

idx2label = {v: k for k, v in train_dataset.label_map.items()}

# Escoger los ejemplos con mayor atención máxima
max_attn = [max([max(tweet_scores) for tweet_scores in doc]) for doc in train_scores]
maxi = np.flip(np.argsort(max_attn))

# Mostrar ejemplos
max_examples = min(3, len(train_words), len(train_scores), len(train_pred))
for j in range(max_examples):
    i = maxi[j]
    s = colorize_words_per_tweet(train_words[i], train_scores[i])
    predicted_label = idx2label[train_pred[i]]
    print(f'Predicción: {predicted_label}')
    display(HTML(s))


Predicción: peru


/tmp/ipykernel_9089/4251296781.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = matplotlib.cm.get_cmap('Reds')


Predicción: colombia


Predicción: chile


Se ven resaltadas palabras que, a primera vista, no dan un gran contexto sobre el origen del autor.

## A nivel oracion

In [94]:
from IPython.display import display, HTML
import matplotlib
import matplotlib.pyplot as plt

def colorize_tweets(tweet_list, attn_weights):
    cmap = matplotlib.cm.get_cmap('Reds')
    template = '<div style="background-color: {}; padding:4px; margin:2px">{}</div>'
    colored_output = ''
    for tweet, weight in zip(tweet_list, attn_weights):
        color = matplotlib.colors.rgb2hex(cmap(weight)[:3])
        text = ' '.join(tweet)
        colored_output += template.format(color, text)
    return colored_output

idx2label = {v: k for k, v in train_dataset.label_map.items()}

# Escoger los ejemplos con mayor atención máxima
max_attn = [np.max(scores) for scores in train_scores]
maxi = np.flip(np.argsort(max_attn))

# Mostrar los ejemplos seguros
max_examples = min(2, len(train_words), len(train_scores), len(train_pred))
for j in range(max_examples):
    i = maxi[j]
    s = colorize_tweets(train_words[i], train_scores[i][:len(train_words[i])])
    predicted_label = idx2label[train_pred[i]]
    print(f'Predicción: {predicted_label}')
    display(HTML(s))

Predicción: peru


/tmp/ipykernel_9089/782181285.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = matplotlib.cm.get_cmap('Reds')


Predicción: chile


Sin embargo, a nivel oracion se resaltan tweets que dan mucho contexto sobre su origen, como la mencion de ciudades del pais de origen.

# Preguntas
1. ¿Qué representación de términos uso el profesor para participar?
    - Se usa una Bag-of-Terms que captura la ocurrencia de las palabras en el espacio de usuario-documentos
2. ¿Qué usó el primer lugar de la competencia?
    - Uso una SVM lineal con unigramas de palabras y 3-, 4- y 5-gramas
3. ¿Cuántos y que competidores usaron deep learning? En una o dos oraciones escriba qué hicieron
    - Los autores de [29] usaron RNN; los de [56, 58] usaron una CNN (red convolucional); los de [41] usaron ambos tipos de NN ademas de usar un mecanismo de atencion, una capa de max-pooling y una capa completamente conexa; por su parte, los autores de [45] usaron regresion logistica combinada con un proceso gaussiano entrenado con embeddings de palabras; finalmente, los de [18] usaron Deep Averaging Networks